In [1]:
# Cell 1: Install necessary libraries
!pip install langchain langchain-core langchain-community langgraph gradio

In [2]:
# Cell 2: Imports and Ollama LLM Setup

import os
from typing import TypedDict, List
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate

# NEW: Import for Ollama
from langchain_community.chat_models import ChatOllama
from langgraph.graph import StateGraph, END

# --- Initialize the Local Language Model (Ollama) ---
# IMPORTANT:
# 1. Ensure Ollama is running in your terminal BEFORE starting Jupyter: `ollama serve`
# 2. Ensure you have downloaded your desired model: `ollama pull llama3` (or mistral, phi3 etc.)
# 3. Replace 'llama3' below with the actual model name you pulled if different.
llm = ChatOllama(model="llama3", temperature=0.7) # Added temperature for a bit more creativity
print(f"Local LLM (Ollama model: {llm.model}) initialized.")

# --- Quick LLM Test ---
try:
    print("\n--- Testing Ollama LLM connection ---")
    response = llm.invoke("Briefly explain the concept of multi-agent systems in AI.")
    print(f"LLM Test Response: {response.content[:150]}...") # Print first 150 chars
    print("--- Ollama LLM connection successful! ---")
except Exception as e:
    print(f"\nERROR: Could not connect to Ollama LLM or model not found: {e}")
    print("Please ensure:")
    print("1. Ollama server is running (open a terminal and type `ollama serve`).")
    print("2. You have pulled the specified model (e.g., `ollama pull llama3`).")
    print("3. The model name in ChatOllama matches the one you pulled.")

C:\Users\momin\AppData\Local\Temp\ipykernel_23208\1814625557.py:17: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = ChatOllama(model="llama3", temperature=0.7) # Added temperature for a bit more creativity


Local LLM (Ollama model: llama3) initialized.

--- Testing Ollama LLM connection ---
LLM Test Response: In Artificial Intelligence (AI), a Multi-Agent System (MAS) refers to a system where multiple autonomous agents interact and cooperate with each other...
--- Ollama LLM connection successful! ---


In [3]:
# Cell 3: Define Planner State and Enhanced Node Functions

class PlannerState(TypedDict):
    messages: List[BaseMessage]
    city: str
    interests: List[str]
    itinerary: str

# --- Define Agent Functions (Nodes) ---
# These functions will be called by LangGraph

def input_city_node(state: PlannerState):
    """Placeholder for the 'get city' step."""
    print("--- Agent: Preparing for City Input ---")
    # In a real multi-agent system, this might involve an agent prompting for city.
    # For this Gradio app, the city comes directly from the UI.
    return state

def input_interests_node(state: PlannerState):
    """Placeholder for the 'get interests' step."""
    print(f"--- Agent: Preparing for Interests Input for {state.get('city')} ---")
    # Similarly, interests come from the UI.
    return state

def create_itinerary_node(state: PlannerState):
    """Uses the LLM to generate a travel itinerary."""
    print("--- Agent: Generating Itinerary with Local LLM ---")
    city = state["city"]
    interests = ", ".join(state["interests"]) if state["interests"] else "general sightseeing"

    # --- ENHANCED PROMPT FOR STRUCTURED OUTPUT ---
    prompt_template = ChatPromptTemplate.from_messages([
        ("system", (
            "You are an expert travel planner. Your task is to create a detailed 3-day travel itinerary "
            "for the given city and interests. The itinerary MUST be structured clearly with specific "
            "daily labels and time-based activities (e.g., Morning, Afternoon, Evening). "
            "Include specific suggestions for attractions, food, and activities relevant to the interests. "
            "Ensure the output is easy to read and follows a consistent format."
        )),
        ("user", (
            f"Plan a 3-day trip to {city} with the following interests: {interests}. "
            "Format the itinerary as follows:\n\n"
            "**Day 1: [Theme/Area for Day 1]**\n"
            "- Morning: [Activity 1]\n"
            "- Afternoon: [Activity 2]\n"
            "- Evening: [Activity 3]\n\n"
            "**Day 2: [Theme/Area for Day 2]**\n"
            "- Morning: [Activity 1]\n"
            "- Afternoon: [Activity 2]\n"
            "- Evening: [Activity 3]\n\n"
            "**Day 3: [Theme/Area for Day 3]**\n"
            "- Morning: [Activity 1]\n"
            "- Afternoon: [Activity 2]\n"
            "- Evening: [Activity 3]\n\n"
            "Include practical tips if helpful (e.g., best way to get around)."
        ))
    ])
    # --- END ENHANCED PROMPT ---

    chain = prompt_template | llm
    try:
        response = chain.invoke({"city": city, "interests": interests})
        itinerary = response.content
        print("\n--- Agent: Itinerary Generated by Local LLM! ---")
    except Exception as e:
        itinerary = (f"Error generating itinerary with local LLM: {e}\n"
                     "Please ensure Ollama is running and the model is loaded correctly.")
        print(f"\n--- Agent: Error during Itinerary Generation ---")

    return {"itinerary": itinerary, "messages": state.get("messages", []) + [AIMessage(content=itinerary)]}

print("PlannerState and Node functions defined.")

PlannerState and Node functions defined.


In [4]:
# Cell 4: Create and Compile the LangGraph Graph

workflow = StateGraph(PlannerState)

workflow.add_node("get_city", input_city_node)
workflow.add_node("get_interests", input_interests_node)
workflow.add_node("generate_itinerary", create_itinerary_node)

workflow.set_entry_point("get_city")

workflow.add_edge("get_city", "get_interests")
workflow.add_edge("get_interests", "generate_itinerary")
workflow.add_edge("generate_itinerary", END)

app = workflow.compile()

print("LangGraph workflow compiled.")

LangGraph workflow compiled.


In [5]:
# Cell 5: Run the Graph (Interactive Test with Local LLM)

# Define your desired city and interests for this test run
test_city = "Kyoto"
test_interests = ["temples", "gardens", "traditional culture", "food"]

# Initialize the state for this run
initial_state = {
    "messages": [HumanMessage(content=f"User wants to visit {test_city} with interests: {', '.join(test_interests)}")],
    "city": test_city,
    "interests": test_interests,
    "itinerary": ""
}

print(f"Running planner for City: {test_city}, Interests: {', '.join(test_interests)}")
print("Please be patient, local LLM inference can take some time depending on your hardware.")

# Invoke the graph with the initial state
final_state = app.invoke(initial_state)

print("\n--- Final Itinerary ---")
print(final_state["itinerary"])

Running planner for City: Kyoto, Interests: temples, gardens, traditional culture, food
Please be patient, local LLM inference can take some time depending on your hardware.
--- Agent: Preparing for City Input ---
--- Agent: Preparing for Interests Input for Kyoto ---
--- Agent: Generating Itinerary with Local LLM ---

--- Agent: Itinerary Generated by Local LLM! ---

--- Final Itinerary ---
Here is a suggested 3-day itinerary for Kyoto:

**Day 1: Fushimi and Southern Kyoto**
- Morning: Visit the iconic Fushimi Inari Shrine (), famous for its thousands of vermilion torii gates. Take the mountain path up to the shrine's summit for stunning views of the city. (9:00 AM - 11:00 AM)
- Afternoon: Explore the nearby Tofuku-ji Temple (), a Zen temple with beautiful gardens and a serene atmosphere. Take a stroll around the adjacent Higashiyama neighborhood, known for its traditional shops and cafes. (11:30 AM - 2:30 PM)
- Evening: Enjoy a traditional Kyoto-style kaiseki dinner at Gion Nanba, a 

In [15]:
# Cell 6: Gradio App Integration with Local LLM (Streaming Output)

import gradio as gr
# ... (rest of your imports and code from previous cells are assumed to be run) ...

# Make sure `llm` is defined from Cell 2 and `app` from Cell 4.

def stream_planner_output(city_input: str, interests_input: str):
    if not city_input:
        yield "Please enter a city."
        return

    processed_interests = [interest.strip() for interest in interests_input.split(',') if interest.strip()]

    # --- RE-USING THE ENHANCED PROMPT ---
    prompt_template = ChatPromptTemplate.from_messages([
        ("system", (
            "You are an expert travel planner. Your task is to create a detailed 3-day travel itinerary "
            "for the given city and interests. The itinerary MUST be structured clearly, with specific "
            "daily labels and time-based activities (e.g., Morning, Afternoon, Evening). "
            "Include specific suggestions for attractions, food, and activities relevant to the interests. "
            "Ensure the output is easy to read and follows a consistent format."
        )),
        ("user", (
            f"Plan a 3-day trip to {city_input} with the following interests: {', '.join(processed_interests)}. "
            "Format the itinerary as follows:\n\n"
            "**Day 1: [Theme/Area for Day 1]**\n"
            "- Morning: [Activity 1]\n"
            "- Afternoon: [Activity 2]\n"
            "- Evening: [Activity 3]\n\n"
            "**Day 2: [Theme/Area for Day 2]**\n"
            "- Morning: [Activity 1]\n"
            "- Afternoon: [Activity 2]\n"
            "- Evening: [Activity 3]\n\n"
            "**Day 3: [Theme/Area for Day 3]**\n"
            "- Morning: [Activity 1]\n"
            "- Afternoon: [Activity 2]\n"
            "- Evening: [Activity 3]\n\n"
            "Include practical tips if helpful (e.g., best way to get around)."
        ))
    ])
    # --- END ENHANCED PROMPT ---

    chain = prompt_template | llm

    current_itinerary_text = ""
    yield "Generating your personalized itinerary... This might take a moment with a local LLM."

    try:
        for chunk in chain.stream({"city": city_input, "interests": ', '.join(processed_interests)}):
            content = chunk.content
            if content:
                current_itinerary_text += content
                yield current_itinerary_text
    except Exception as e:
        yield f"An error occurred during itinerary generation: {e}\nEnsure Ollama is running and the model is available."


# --- Create the Gradio Interface ---
iface = gr.Interface(
    fn=stream_planner_output,
    inputs=[
        gr.Textbox(label="City (e.g., Tokyo, Rome)", placeholder="Enter your destination city"),
        gr.Textbox(label="Interests (comma-separated, e.g., food, history, art)", placeholder="museums, parks, shopping")
    ],
    outputs=gr.Markdown(label="Your Travel Itinerary"),
    title="Multi-Agent Travel Planner",
    description="Plan your next adventure with a local AI agent! Enter your city and interests to get a personalized 3-day itinerary.",
    examples=[
        ["Sydney", "beaches, opera, nature"],
        ["Rome", "ancient ruins, pizza, art"],
        ["Amsterdam", "canals, cycling, museums"]
    ],
    allow_flagging="never"
)

# Launch the Gradio app directly within the Jupyter Notebook
if __name__ == "__main__":
    # This is the line that creates the public URL:
    iface.launch(inbrowser=True, share=True)

C:\Users\momin\anaconda3\Lib\site-packages\gradio\interface.py:416: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7862

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.
